In [1]:
"""
Historical Price Data with ADV Filter using LSEG Data Packages

This script fetches historical price and volume data, then filters stocks
based on 3-month Average Daily Volume (ADV) >= 5M USD.

ADV formula: ADV_t = (1/60) * Σ(USD_Volume_{t-i}) for i=1 to 60
"""

import lseg.data as ld
import pandas as pd
from typing import Tuple, List
import numpy as np
from datetime import datetime, timedelta
import json

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

import warnings

warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path

BASE_DIR = Path("/home/jovyan/Daily_new_data_russel")
BASE_DIR.mkdir(parents=True, exist_ok=True)

print("Saving to:", BASE_DIR.resolve())

Saving to: /home/jovyan/Daily_new_data_russel


In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x7f4eff535040 {name='codebook'}>

In [4]:
russel = ld.get_data("0#.RUA",fields = 'TR.CommonName')
russel.head()

,Instrument,Company Common Name
0,RIGL.OQ,Rigel Pharmaceuticals Inc
1,NIC.N,Nicolet Bankshares Inc
2,EHAB.N,Enhabit Inc
3,UNP.N,Union Pacific Corp
4,AON.N,Aon PLC


In [5]:
rics = russel["Instrument"].astype(str).tolist()

In [6]:
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = '2026-02-12'

In [7]:
def get_historical_data(rics, start_date, end_date, fields=None, batch_size=50):
    """
    Fetch historical price and volume data in batches (single-core)
    
    Parameters:
    -----------
    rics : list
        List of Reuters Instrument Codes (RICs)
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format
    fields : list, optional
        Fields to retrieve. Default includes price and volume fields.
    batch_size : int, optional
        Number of RICs to process per batch (default: 50)
    
    Returns:
    --------
    pd.DataFrame
        Historical data with multi-index (Date, RIC)
    """
    if fields is None:
        fields = [
            'TR.PriceClose',           # Closing price
            #'TR.PriceOpen',            # Opening price
            #'TR.PriceHigh',            # High price
            #'TR.PriceLow',             # Low price
            'TR.Volume',               # Trading volume (shares)
            'TR.PriceClose.currency',   # Currency for price
            'TR.TotalReturn1D',
            #'TR.TRBCEconomicSector' # static object process later
            
        ]
    
    total_rics = len(rics)
    num_batches = (total_rics + batch_size - 1) // batch_size
    
    print(f"\nFetching data for {total_rics} instruments...")
    print(f"Date range: {end_date}")
    print(f"Processing in {num_batches} batches of ~{batch_size} RICs each (single-core)\n")
    
    all_data = []
    failed_batches = []
    
    try:
        
        for batch_num in range(num_batches):
            # Get batch of RICs
            batch_start = batch_num * batch_size
            batch_end = min((batch_num + 1) * batch_size, total_rics)
            batch_rics = rics[batch_start:batch_end]

            
            try:
                # Fetch batch data
                df_batch = ld.get_history(
                    universe=batch_rics,
                    fields=fields,
                    start=start_date,
                    end=end_date,
                    interval='daily',
                    #count = look_back_days,
                    parameters = {'Curn': 'USD'}
                )
                
                if not df_batch.empty:
                    df_batch = df_batch.stack(level=0)   # stack RICs
                    all_data.append(df_batch)
                    print("✓")
                else:
                    print("✗ (no data)")
                


                progress = (batch_end / total_rics) * 100
                print(f"Progress: {progress:5.1f}% | Batch {batch_num+1}/{num_batches}s")
                        
            except Exception as batch_error:
                failed_batches.append(batch_num + 1)
                print(f"\n⚠ Warning: Batch {batch_num+1} failed: {batch_error}")

        
        # Combine all batches
        if all_data:
            df_combined = pd.concat(all_data)
            df_combined.index.names = ["Date", "RIC"]
            
            #print(f"\n✓ Retrieved {len(df_combined):,} data points in {elapsed:.1f}s")
            #print(f"  Unique instruments: {df_combined['Instrument'].nunique()}")
            #print(f"  Date range: {df_combined['Date'].min()} to {df_combined['Date'].max()}")
            
            if failed_batches:
                print(f"  ⚠ Failed batches: {failed_batches}")
            
            return df_combined
        else:
            print(f"\n✗ No data retrieved")
            return None
            
    except Exception as e:
        print(f"\n✗ Error fetching historical data: {e}")
        return None


In [8]:
import json

with open("rics_20260214_ADVfiltered.json", "r", encoding="utf-8") as f:
    kept_rics = json.load(f)

In [9]:
df = get_historical_data(rics, start_date, end_date, fields = None)

df_final = df.reset_index()
df_final.to_csv(BASE_DIR /"lseg_historyprice_data_now_to_20260212.csv", index=False, encoding='utf-8-sig')


Fetching data for 2938 instruments...
Date range: 2026-02-21
Processing in 59 batches of ~50 RICs each (single-core)

✓
Progress:   1.7% | Batch 1/59s
✓
Progress:   3.4% | Batch 2/59s
✓
Progress:   5.1% | Batch 3/59s
✓
Progress:   6.8% | Batch 4/59s
✓
Progress:   8.5% | Batch 5/59s
✓
Progress:  10.2% | Batch 6/59s
✓
Progress:  11.9% | Batch 7/59s
✓
Progress:  13.6% | Batch 8/59s
✓
Progress:  15.3% | Batch 9/59s
✓
Progress:  17.0% | Batch 10/59s
✓
Progress:  18.7% | Batch 11/59s
✓
Progress:  20.4% | Batch 12/59s
✓
Progress:  22.1% | Batch 13/59s
✓
Progress:  23.8% | Batch 14/59s
✓
Progress:  25.5% | Batch 15/59s
✓
Progress:  27.2% | Batch 16/59s
✓
Progress:  28.9% | Batch 17/59s
✓
Progress:  30.6% | Batch 18/59s
✓
Progress:  32.3% | Batch 19/59s
✓
Progress:  34.0% | Batch 20/59s
✓
Progress:  35.7% | Batch 21/59s
✓
Progress:  37.4% | Batch 22/59s
✓
Progress:  39.1% | Batch 23/59s
✓
Progress:  40.8% | Batch 24/59s
✓
Progress:  42.5% | Batch 25/59s
✓
Progress:  44.2% | Batch 26/59s
✓
Prog

In [10]:
df = get_historical_data(kept_rics, start_date, end_date, fields = None)

df_final = df.reset_index()
df_final.to_csv(BASE_DIR /"lseg_historyprice_data_now_to_20260212_ADVfiltered.csv", index=False, encoding='utf-8-sig')


Fetching data for 1849 instruments...
Date range: 2026-02-21
Processing in 37 batches of ~50 RICs each (single-core)

✓
Progress:   2.7% | Batch 1/37s
✓
Progress:   5.4% | Batch 2/37s
✓
Progress:   8.1% | Batch 3/37s
✓
Progress:  10.8% | Batch 4/37s
✓
Progress:  13.5% | Batch 5/37s
✓
Progress:  16.2% | Batch 6/37s
✓
Progress:  18.9% | Batch 7/37s
✓
Progress:  21.6% | Batch 8/37s
✓
Progress:  24.3% | Batch 9/37s
✓
Progress:  27.0% | Batch 10/37s
✓
Progress:  29.7% | Batch 11/37s
✓
Progress:  32.4% | Batch 12/37s
✓
Progress:  35.2% | Batch 13/37s
✓
Progress:  37.9% | Batch 14/37s
✓
Progress:  40.6% | Batch 15/37s
✓
Progress:  43.3% | Batch 16/37s
✓
Progress:  46.0% | Batch 17/37s
✓
Progress:  48.7% | Batch 18/37s
✓
Progress:  51.4% | Batch 19/37s
✓
Progress:  54.1% | Batch 20/37s
✓
Progress:  56.8% | Batch 21/37s
✓
Progress:  59.5% | Batch 22/37s
✓
Progress:  62.2% | Batch 23/37s
✓
Progress:  64.9% | Batch 24/37s
✓
Progress:  67.6% | Batch 25/37s
✓
Progress:  70.3% | Batch 26/37s
✓
Prog

In [11]:
def get_historical_data_pe(rics, start_date, end_date, fields=None, batch_size=50):
    """
    Fetch historical price and volume data in batches (single-core)
    
    Parameters:
    -----------
    rics : list
        List of Reuters Instrument Codes (RICs)
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format
    fields : list, optional
        Fields to retrieve. Default includes price and volume fields.
    batch_size : int, optional
        Number of RICs to process per batch (default: 50)
    
    Returns:
    --------
    pd.DataFrame
        Historical data with multi-index (Date, RIC)
    """
    if fields is None:
        fields = [
            #'TR.PriceClose',           # Closing price
            #'TR.PriceOpen',            # Opening price
            #'TR.PriceHigh',            # High price
            #'TR.PriceLow',             # Low price
            #'TR.Volume',               # Trading volume (shares)
            #'TR.PriceClose.currency',   # Currency for price
            #'TR.TotalReturn1D',
            #'TR.TRBCEconomicSector' # static object process later
            "TR.PE",
            
        ]
    
    total_rics = len(rics)
    num_batches = (total_rics + batch_size - 1) // batch_size
    
    print(f"\nFetching data for {total_rics} instruments...")
    print(f"Date range: {end_date}")
    print(f"Processing in {num_batches} batches of ~{batch_size} RICs each (single-core)\n")
    
    all_data = []
    failed_batches = []
    
    try:
        
        for batch_num in range(num_batches):
            # Get batch of RICs
            batch_start = batch_num * batch_size
            batch_end = min((batch_num + 1) * batch_size, total_rics)
            batch_rics = rics[batch_start:batch_end]

            
            try:
                # Fetch batch data
                df_batch = ld.get_history(
                    universe=batch_rics,
                    fields=fields,
                    start=start_date,
                    end=end_date,
                    interval='daily',
                    #count = look_back_days,
                    parameters = {'Curn': 'USD'}
                )
                
                if not df_batch.empty:
                    df_batch = df_batch.stack(level=0)   # stack RICs
                    all_data.append(df_batch)
                    print("✓")
                else:
                    print("✗ (no data)")
                


                progress = (batch_end / total_rics) * 100
                print(f"Progress: {progress:5.1f}% | Batch {batch_num+1}/{num_batches}s")
                        
            except Exception as batch_error:
                failed_batches.append(batch_num + 1)
                print(f"\n⚠ Warning: Batch {batch_num+1} failed: {batch_error}")

        
        # Combine all batches
        if all_data:
            df_combined = pd.concat(all_data)
            df_combined.index.names = ["Date", "RIC"]
            
            # Rename column to a clean title
            #df_combined.columns = ["Price to Earning"]
            
            #print(f"\n✓ Retrieved {len(df_combined):,} data points in {elapsed:.1f}s")
            #print(f"  Unique instruments: {df_combined['Instrument'].nunique()}")
            #print(f"  Date range: {df_combined['Date'].min()} to {df_combined['Date'].max()}")
            
            if failed_batches:
                print(f"  ⚠ Failed batches: {failed_batches}")
            
            return df_combined
        else:
            print(f"\n✗ No data retrieved")
            return None
            
    except Exception as e:
        print(f"\n✗ Error fetching historical data: {e}")
        return None

In [12]:
df = get_historical_data(rics, start_date, end_date, fields = "TR.PE")


if isinstance(df, pd.Series):
    df = df.to_frame(name="Price to Earning")

    
df_final = df.reset_index()


df_final.to_csv(BASE_DIR /"lseg_Price-Earning_data_now_to_20260212.csv", index = False, encoding="utf-8-sig")


Fetching data for 2938 instruments...
Date range: 2026-02-21
Processing in 59 batches of ~50 RICs each (single-core)

✓
Progress:   1.7% | Batch 1/59s
✓
Progress:   3.4% | Batch 2/59s
✓
Progress:   5.1% | Batch 3/59s
✓
Progress:   6.8% | Batch 4/59s
✓
Progress:   8.5% | Batch 5/59s
✓
Progress:  10.2% | Batch 6/59s
✓
Progress:  11.9% | Batch 7/59s
✓
Progress:  13.6% | Batch 8/59s
✓
Progress:  15.3% | Batch 9/59s
✓
Progress:  17.0% | Batch 10/59s
✓
Progress:  18.7% | Batch 11/59s
✓
Progress:  20.4% | Batch 12/59s
✓
Progress:  22.1% | Batch 13/59s
✓
Progress:  23.8% | Batch 14/59s
✓
Progress:  25.5% | Batch 15/59s
✓
Progress:  27.2% | Batch 16/59s
✓
Progress:  28.9% | Batch 17/59s
✓
Progress:  30.6% | Batch 18/59s
✓
Progress:  32.3% | Batch 19/59s
✓
Progress:  34.0% | Batch 20/59s
✓
Progress:  35.7% | Batch 21/59s
✓
Progress:  37.4% | Batch 22/59s
✓
Progress:  39.1% | Batch 23/59s
✓
Progress:  40.8% | Batch 24/59s
✓
Progress:  42.5% | Batch 25/59s
✓
Progress:  44.2% | Batch 26/59s
✓
Prog

In [13]:
df = get_historical_data(kept_rics, start_date, end_date, fields = None)


if isinstance(df, pd.Series):
    df = df.to_frame(name="Price to Earning")

    
df_final = df.reset_index()


df_final.to_csv(BASE_DIR /"lseg_Price-Earning_data_now_to_20260212_ADVfiltered.csv", index = False, encoding="utf-8-sig")


Fetching data for 1849 instruments...
Date range: 2026-02-21
Processing in 37 batches of ~50 RICs each (single-core)

✓
Progress:   2.7% | Batch 1/37s
✓
Progress:   5.4% | Batch 2/37s
✓
Progress:   8.1% | Batch 3/37s
✓
Progress:  10.8% | Batch 4/37s
✓
Progress:  13.5% | Batch 5/37s
✓
Progress:  16.2% | Batch 6/37s
✓
Progress:  18.9% | Batch 7/37s
✓
Progress:  21.6% | Batch 8/37s
✓
Progress:  24.3% | Batch 9/37s
✓
Progress:  27.0% | Batch 10/37s
✓
Progress:  29.7% | Batch 11/37s
✓
Progress:  32.4% | Batch 12/37s
✓
Progress:  35.2% | Batch 13/37s
✓
Progress:  37.9% | Batch 14/37s
✓
Progress:  40.6% | Batch 15/37s
✓
Progress:  43.3% | Batch 16/37s
✓
Progress:  46.0% | Batch 17/37s
✓
Progress:  48.7% | Batch 18/37s
✓
Progress:  51.4% | Batch 19/37s
✓
Progress:  54.1% | Batch 20/37s
✓
Progress:  56.8% | Batch 21/37s
✓
Progress:  59.5% | Batch 22/37s
✓
Progress:  62.2% | Batch 23/37s
✓
Progress:  64.9% | Batch 24/37s
✓
Progress:  67.6% | Batch 25/37s
✓
Progress:  70.3% | Batch 26/37s
✓
Prog

In [16]:
#df = get_historical_data(".SPX", start_date, end_date, fields = None)

df = ld.get_history(
    universe=".SPX",
    fields="TR.PriceClose",
    interval="daily",
    start=start_date,
    end=end_date,
)

if isinstance(df, pd.Series):
    df = df.to_frame(name="Close Price")

df_final.to_csv(BASE_DIR /"lseg_historyprice_S&P500_now_to_20260212.csv", index=False, encoding='utf-8-sig')

In [ ]:
def calculate_usd_volume(df):
    """
    Calculate USD trading volume for each (Date, RIC)

    USD_Volume = Volume * Price Close * FX_Rate (if not USD)

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain:
        ['Volume', 'Price Close', 'Currency']

    Returns:
    --------
    pd.DataFrame
        With added 'USD_Volume' column
    """
    print("\nCalculating USD volume...")

    df = df.copy()

    # Sanity check
    required_cols = {"Volume", "Price Close", "Currency"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # Base calculation (assume USD)
    df["USD_Volume"] = df["Volume"] * df["Price Close"]

    # Identify non-USD rows
    non_usd = df["Currency"] != "USD"
    non_usd_df = df.loc[df["Currency"].ne("USD")].copy()


    if non_usd.any():
        print(f"⚠ {non_usd.sum()} rows are non-USD (FX rate = 1.0 placeholder)")
        # Placeholder FX rate
        df.loc[non_usd, "USD_Volume"] *= 1.0

    return df, non_usd_df



In [ ]:
def calculate_adv(df, lookback_days=75, min_coverage=0.8):
    print(f"\nCalculating {lookback_days}-day ADV for each stock...")

    adv_records = []

    for ric, g in df.groupby(level="RIC"):
        g = g.sort_index(level="Date")

        recent = g.tail(lookback_days)
        n_obs = recent["USD_Volume"].notna().sum()

        if n_obs >= lookback_days * min_coverage:
            adv_records.append({
                "RIC": ric,
                "ADV_USD": recent["USD_Volume"].mean(),
                "Days_Used": n_obs,
                "Latest_Date": recent.index.get_level_values("Date")[-1]
            })
        else:
            print(f"  Warning: Insufficient data for {ric} ({n_obs} days)")

    return pd.DataFrame(adv_records)


In [ ]:
def apply_single_stock_limits(
    adv_summary: pd.DataFrame,
    threshold_usd: float = 5_000_000,
    adv_col: str = "ADV_USD",
    ric_col: str = "RIC",
    pct_of_adv: float = 0.025,
    max_usd_cap: float = 2_000_000,
    limit_col: str = "Daily_Limit_USD",
) -> Tuple[pd.DataFrame, pd.DataFrame, List[str]]:
    """
    Step 1: Screen by ADV >= threshold_usd (e.g. 5M).
    Step 2: For kept names, compute Daily_Limit_USD = min(pct_of_adv * ADV, max_usd_cap).

    Returns:
        filtered_adv: kept rows with Daily_Limit_USD added
        excluded_adv: excluded rows (no daily limit computed)
        kept_rics: list of kept RICs
    """
    df = adv_summary.copy()

    required = {adv_col, ric_col}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    df[adv_col] = pd.to_numeric(df[adv_col], errors="coerce")

    # Step 1: ADV screen
    mask_keep = df[adv_col] >= threshold_usd
    filtered_adv = df.loc[mask_keep].copy()
    excluded_adv = df.loc[~mask_keep].copy()
    kept_rics = filtered_adv[ric_col].dropna().tolist()

    # Step 2: compute daily trading limit only for kept
    filtered_adv[limit_col] = (filtered_adv[adv_col] * pct_of_adv).clip(upper=max_usd_cap)


    return filtered_adv, excluded_adv, kept_rics